In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm

In [ ]:
def gen_mask(length, p, prob_func='constant'):
    """
    Call of prob_func gives 1 or 0 with certain probability.
    """
    mask_mat = np.zeros((length, length))
    for i in range(length):
        for j in range(length):
            if i >= j:
                val = 0
                r = np.random.uniform()
                prob = 0
                if prob_func == 'pn':
                    prob = p
                if prob_func == 'log':
                    prob = p * np.log(length) / length
                if prob_func == 'p':
                    prob = p / length
                if r <= prob:
                    val = 1
                mask_mat[i, j] = val
                mask_mat[j, i] = val
    return mask_mat


def get_spectrum(n, theta, mask):
    G = np.random.normal(size=(n, n))
    X = (G + np.transpose(G)) / (2*n)**0.5
    u = np.random.rand(n)
    u = u / np.linalg.norm(u)
    u = u[None, :]
    X_tilda = X + theta * u * u.transpose()
    X_bar = X_tilda * mask
    eigvals, eigvecs = np.linalg.eig(X_bar)
    idx = np.argmax(eigvals)
    max_eigvec = eigvecs[:, idx]
    inner_prod = np.inner(max_eigvec, u.reshape(-1))
    return sorted(eigvals), inner_prod

# Eigen-spectrum of $\bar{X}$

## d = p * n

The first thing I want to check is how p changes the spectrum. So I selected p from 0 to 1 and plotted the spectrums.

In [ ]:
n = 100
theta = 100
ps = [0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1]
eigvals_list = []
ip_list = []
for p in ps:
    mask = gen_mask(n, p, 'pn')
    eigvals, inner_prod = get_spectrum(n, theta, mask)
    eigvals_list.append(eigvals)
    ip_list.append(inner_prod)

In [ ]:
for i in range(len(ps)):
    plt.plot(eigvals_list[i], 'x', label=f'{ps[i]}')
plt.legend()
plt.show()

It seems like the largest eigenvalues is propotional to p. And the range of the other eigenvalues increases and then decreases. To check the first hypothesis, I plotted max(eigvalues) v.s. p.

In [ ]:
largest_eigvals = []
for eigvals in eigvals_list:
    largest_eigvals.append(np.max(eigvals))
plt.plot(ps, largest_eigvals)
plt.xlabel('p')
plt.ylabel('largest eigenvalue')
plt.show()

It looks like the largest eigenvalues are propotional to p. However, the curve is definitely not a straight line. I further checked whether these nonlinear parts are actually noise.

In [ ]:
n = 100
theta = 100
ps = [0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1]
trials = 10
for _ in range(trials):
    eigvals_list = []
    ip_list = []
    for p in ps:
        mask = gen_mask(n, p, 'pn')
        eigvals, inner_prod = get_spectrum(n, theta, mask)
        eigvals_list.append(eigvals)
        ip_list.append(inner_prod)
    largest_eigvals = []
    for eigvals in eigvals_list:
        largest_eigvals.append(np.max(eigvals))
    plt.plot(ps, largest_eigvals)
plt.xlabel('p')
plt.ylabel('largest eigenvalue')
plt.show()

How about the other two probability functions?

## d = p * log(n)

In [ ]:
n = 100
theta = 100
ps = [0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1]
trials = 10
for _ in range(trials):
    eigvals_list = []
    ip_list = []
    for p in ps:
        mask = gen_mask(n, p, 'log')
        eigvals, inner_prod = get_spectrum(n, theta, mask)
        eigvals_list.append(eigvals)
        ip_list.append(inner_prod)
    largest_eigvals = []
    for eigvals in eigvals_list:
        largest_eigvals.append(np.real(np.max(eigvals)))
    plt.plot(ps, largest_eigvals)
plt.xlabel('p')
plt.ylabel('largest eigenvalue')
plt.show()

Looks noisy. More trials and take mean value and larger n.

In [ ]:
n = 1000
theta = 100
ps = [0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1]
trials = 5

mean_largest_eigvals = np.zeros(len(ps))
for _ in tqdm(range(trials)):
    eigvals_list = []
    ip_list = []
    for p in ps:
        mask = gen_mask(n, p, 'log')
        eigvals, inner_prod = get_spectrum(n, theta, mask)
        eigvals_list.append(eigvals)
        ip_list.append(inner_prod)
    largest_eigvals = []
    for eigvals in eigvals_list:
        largest_eigvals.append(np.real(np.max(eigvals)))
    mean_largest_eigvals += np.array(largest_eigvals) / len(ps)
plt.plot(ps, mean_largest_eigvals)
plt.xlabel('p')
plt.ylabel('mean largest eigenvalue')
plt.show()

## d = p

In [ ]:
n = 1000
theta = 100
ps = [0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1]
trials = 5

mean_largest_eigvals = np.zeros(len(ps))
for _ in tqdm(range(trials)):
    eigvals_list = []
    ip_list = []
    for p in ps:
        mask = gen_mask(n, p, 'p')
        eigvals, inner_prod = get_spectrum(n, theta, mask)
        eigvals_list.append(eigvals)
        ip_list.append(inner_prod)
    largest_eigvals = []
    for eigvals in eigvals_list:
        largest_eigvals.append(np.real(np.max(eigvals)))
    mean_largest_eigvals += np.array(largest_eigvals) / len(ps)
plt.plot(ps, mean_largest_eigvals)
plt.xlabel('p')
plt.ylabel('mean largest eigenvalue')
plt.show()

Still noisy. Is $\theta$ too small? Test with other values.

# Inner product between u and leading eigenvector of $\bar{X}$

In [ ]:
def get_innerprods(trials, n, theta, ps, prob_func):
    ips = np.zeros(len(ps))
    for _ in tqdm(range(trials)):
        ip_list = []
        for p in ps:
            mask = gen_mask(n, p, prob_func=prob_func)
            _, inner_prod = get_spectrum(n, theta, mask)
            ip_list.append(abs(inner_prod))
        ips += np.array(ip_list) / trials
    return ips
ps = np.linspace(0, 1, 30)
thetas = [1, 10, 100, 1000]

## d = p * n

In [ ]:
for theta in thetas:
    ips = get_innerprods(trials=10, n=100, theta=100, ps=ps, prob_func='pn')
    plt.plot(ps, ips, label=f'{theta}')
plt.legend()
plt.show()

In [ ]:
trials = 100
n = 200
theta = 100
ps = np.linspace(0, 1, 30)
ips = np.zeros(len(ps))
for _ in tqdm(range(trials)):
    ip_list = []
    for p in ps:
        mask = gen_mask(n, p, 'log')
        eigvals, inner_prod = get_spectrum(n, theta, mask)
        ip_list.append(abs(inner_prod))
    ips += np.array(ip_list) / trials
plt.plot(ps, ips)
plt.show()

In [ ]:
trials = 100
n = 200
theta = 100
ps = np.linspace(0, 1, 30)
ips = np.zeros(len(ps))
for _ in tqdm(range(trials)):
    ip_list = []
    for p in ps:
        mask = gen_mask(n, p, 'p')
        eigvals, inner_prod = get_spectrum(n, theta, mask)
        ip_list.append(abs(inner_prod))
    ips += np.array(ip_list) / trials
plt.plot(ps, ips)
plt.show()